# Extract features without having raw_data 

# OnePlus Mobiles😎

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np
from itertools import zip_longest


In [2]:
def bs4_soup(pages):
    URL = f'https://www.flipkart.com/search?q=one+plus+mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={pages}'
    request = requests.get(URL)
    soup = bs(request.text,'html.parser')
    soup = soup.find('div',class_ = '_1YokD2 _3Mn1Gg')

    return soup , URL

In [3]:
def mob_specs(soup):
    names = soup.find_all('div', class_='_4rR01T')
    mob_names_list = [i.text if i else None for i in names]

    ratings = soup.find_all('div', class_='_3LWZlK')
    mob_ratings = [i.text if i else None for i in ratings]

    price = soup.find_all('div', class_='_30jeq3 _1_WHN1')
    mob_price = [i.text if i else None for i in price]

    img = soup.find_all('div', class_='CXW8mj')
    mob_img_URL = [i.find('img')['src'] if i.find('img') else None for i in img]

    # Find the maximum length among all lists
    max_length = max(len(mob_names_list), len(mob_ratings), len(mob_price), len(mob_img_URL))

    # Use zip_longest to ensure all lists have the same length
    mob_names = list(zip_longest(mob_names_list, [None] * max_length,fillvalue=np.nan))
    mob_ratings = list(zip_longest(mob_ratings, [None] * max_length,fillvalue=np.nan))
    mob_price = list(zip_longest(mob_price, [None] * max_length,fillvalue=np.nan))
    mob_img_URL = list(zip_longest(mob_img_URL, [None] * max_length,fillvalue=np.nan))

    return mob_names, mob_ratings, mob_price, mob_img_URL

# mob_names, mob_ratings, mob_price, mob_img_URL = mob_specs(soup)

# print(f"Length of mob_names: {len(mob_names)}")
# print(f"Length of mob_ratings: {len(mob_ratings)}")
# print(f"Length of mob_price: {len(mob_price)}")
# print(f"Length of mob_img_URL: {len(mob_img_URL)}")


In [4]:
import pandas as pd
from itertools import zip_longest

def features_from_mob_links(soup):
    mobile_links = ['https://www.flipkart.com' + i['href'] for i in soup.find_all('a', class_='_1fQZEK')]
    mobile_req = [requests.get(url=i) for i in mobile_links]
    mobile_soup = [bs(i.text, 'html.parser').find('div', class_='_1YokD2 _2GoDe3') for i in mobile_req]
    mobile_specs_soup = [i.find_all('div', class_='_3k-BhJ') for i in mobile_soup]

    storage_ram = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Memory' in str(specs_.text) and 'Storage' in str(specs_.text)]
    os_processor = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Os' in str(specs_.text) and 'Processor' in str(specs_.text)]
    camera = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Camera' in str(specs_.text) and 'FeaturesPrimary' in str(specs_.text)]
    display = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Display' in str(specs_.text) and 'FeaturesDisplay' in str(specs_.text)]
    network = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Connectivity' in str(specs_.text) and 'FeaturesNetwork' in str(specs_.text)]
    battery = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Battery' in str(specs_.text) and 'FeaturesBattery' in str(specs_.text)]

    # Find the maximum length among all lists
    max_length = max(len(storage_ram), len(os_processor), len(camera), len(display), len(network), len(battery))

    # Use zip_longest to ensure all lists have the same length
    storage_ram = list(zip_longest(storage_ram, [None] * max_length,fillvalue=np.nan))
    os_processor = list(zip_longest(os_processor, [None] * max_length,fillvalue=np.nan))
    camera = list(zip_longest(camera, [None] * max_length,fillvalue=np.nan))
    display = list(zip_longest(display, [None] * max_length,fillvalue=np.nan))
    network = list(zip_longest(network, [None] * max_length,fillvalue=np.nan))
    battery = list(zip_longest(battery, [None] * max_length,fillvalue=np.nan))

    return mobile_specs_soup, storage_ram, os_processor, camera, display, network, battery


In [5]:
df = pd.DataFrame()

page_start = 1

for pages in range(page_start, 18):

    soup, URL = bs4_soup(pages=pages)
    mob_names, mob_ratings, mob_price, mob_img_URL = mob_specs(soup)
    mobile_specs_soup, storage_ram, os_processor, camera, display, network, battery = features_from_mob_links(soup=soup)


    if mobile_specs_soup is not None and soup is not None:
        
        print(f"Length of mob_names: {len(mob_names)}")
        print(f"Length of mob_ratings: {len(mob_ratings)}")
        print(f"Length of mob_price: {len(mob_price)}")
        print(f"Length of mob_img_URL: {len(mob_img_URL)}")
        print(f"Length of storage_ram: {len(storage_ram)}")
        print(f"Length of os_processor: {len(os_processor)}")
        print(f"Length of camera: {len(camera)}")
        print(f"Length of display: {len(display)}")
        print(f"Length of network: {len(network)}")
        print(f"Length of battery: {len(battery)}")

        names = [i for i in mob_names]
        ratings = [i for i in mob_ratings]
        price = [i for i in mob_price]
        imgURL = [i for i in mob_img_URL]
        storage_ram_ = [i for i in storage_ram]
        cam = [i for i in camera]
        os_pro = [i for i in os_processor]
        dis = [i for i in display]
        net = [i for i in network]
        bat = [i for i in battery]

        data_dict = {
            'name': names,
            'ratings': ratings,
            'price': price,
            'imgURL': imgURL,
            'storage_ram': storage_ram_,
            'camera': cam,     
            'oS_Processor': os_pro,    
            'display': dis,
            'network': net,     
            'battery': bat,    
        }

        page_df = pd.DataFrame(data=data_dict)

        if not page_df.empty:
            df = pd.concat(objs=[df, page_df], ignore_index=True)
            print(f'scraping data from page:{pages}')
            print(f'working in {URL} Page')
            print(f'page df : page_shape : {page_df.shape} \n {page_df.sample(2)}')
            print(f'df : df shape : {df.shape} \n {df.sample(2)}')
            print()

    else:
        print(f"Skipping page {pages} due to empty mobile_specs_soup or soup.")


Length of mob_names: 24
Length of mob_ratings: 24
Length of mob_price: 24
Length of mob_img_URL: 24
Length of storage_ram: 24
Length of os_processor: 24
Length of camera: 24
Length of display: 24
Length of network: 24
Length of battery: 24
scraping data from page:1
working in https://www.flipkart.com/search?q=one+plus+mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1 Page
page df : page_shape : (24, 10) 
                                                 name      ratings   
19  (OnePlus 8T 5G (Aquamarine Green, 256 GB), None)  (3.3, None)  \
23           (OnePlus Nord (Gray Onyx, 64 GB), None)  (4.3, None)   

              price                                             imgURL   
19  (₹34,999, None)  (https://rukminim2.flixcart.com/image/312/312/...  \
23  (₹24,490, None)  (https://rukminim2.flixcart.com/image/312/312/...   

                                          storage_ram   
19             (Internal Storage256 GBRAM12 GB, None)  \
23  (Inte

In [12]:
df

,name,ratings,price,imgURL,storage_ram,camera,oS_Processor,display,network,battery
0,"(OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)...","(4.4, None)","(₹17,955, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM6 GBMemory Card Slot...,"(Primary Camera64MP Rear Camera, None)",(Operating SystemAndroid 10Processor CoreOcta ...,(Display Size16.74 cm (6.59 inch)Resolution241...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity5000 mAh, None)"
1,"(OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB...","(4.4, None)","(₹17,882, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM6 GBMemory Card Slot...,"(Primary Camera64MP Rear Camera, None)",(Operating SystemAndroid 13Processor CoreOcta ...,(Display Size16.74 cm (6.59 inch)Resolution241...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity5000 mAh, None)"
2,"(OnePlus 11R 5G (Sonic Black, 128 GB), None)","(4.5, None)","(₹39,594, None)",(https://rukminim2.flixcart.com/image/312/312/...,"(Internal Storage128 GBRAM8 GB, None)",(Primary Camera50MP Rear CameraDual Camera Len...,(Operating SystemAndroid Oxygen OxygenOS based...,(Display Size17.02 cm (6.7 inch)Resolution2772...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity5000 mAh, None)"
3,"(OnePlus Nord 2T 5G (Gray Shadow, 128 GB), None)","(4.4, None)","(₹28,729, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM8 GBMemory Card Slot...,"(Primary Camera50MP Rear Camera, None)",(Operating SystemAndroid 13Processor CoreOcta ...,(Display Size17.02 cm (6.7 inch)Resolution2400...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity4500 mAh, None)"
4,"(OnePlus 11R 5G (Galactic Silver, 128 GB), None)","(4.5, None)","(₹39,843, None)",(https://rukminim2.flixcart.com/image/312/312/...,"(Internal Storage128 GBRAM8 GB, None)",(Primary Camera50MP Rear CameraDual Camera Len...,(Operating SystemAndroid OxygenOS based on And...,(Display Size17.02 cm (6.7 inch)Resolution2772...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity5000 mAh, None)"
...,...,...,...,...,...,...,...,...,...,...
206,"(OnePlus 11 5G (Eternal Green, 128 GB), None)","(4.4, None)","(₹51,810, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM8 GBMemory Card Slot...,(Primary Camera50MP Rear CameraDual Camera Len...,(Operating SystemAndroid OxygenOS based on And...,(Display Size17.02 cm (6.7 inch)Resolution3216...,"(Network Type5GSupported Networks5G, None)","(Battery Capacity5000 mAh, None)"
207,"(OnePlus 10R 5G (Sierra Black, 256 GB), None)","(4.2, None)","(₹36,990, None)",(https://rukminim2.flixcart.com/image/312/312/...,"(Internal Storage256 GBRAM12 GB, None)","(Primary Camera50MP Rear Camera, None)",(Operating SystemAndroid 12Primary Clock Speed...,(Display Size17.02 cm (6.7 inch)Resolution1080...,"(Network Type5GSupported Networks5G, 4G VoLTE,...","(Battery Capacity5000 mAh, None)"
208,"(OnePlus 3T (Midnight Black, 128 GB), None)","(4.2, None)","(₹9,999, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM6 GBMemory Card Slot...,(Primary Camera16MP Rear CameraSecondary Camer...,(Operating SystemAndroid Marshmallow 6Processo...,(Display Size13.97 cm (5.5 inch)Resolution1920...,"(Network Type4GSupported Networks4G LTE, None)","(Battery Capacity3400 mAh, None)"
209,"(OnePlus 10 Pro 5G (Emerald Forest, 128 GB), N...","(nan, None)","(₹49,999, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM8 GBMemory Card Slot...,"(Primary Camera48MP Rear Camera, None)",(Operating SystemAndroid 12Processor CoreDual ...,(Display Size17.02 cm (6.7 inch)Resolution3216...,"(Network Type5GSupported Networks5G, 4G VoLTE,...","(Battery Capacity5000 mAh, None)"


In [13]:
print(os.getcwd())
print(os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem'))
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [8]:
df.to_csv(path_or_buf=r'data\raw_data\onePlus_mobile_data.csv')

## Realme Mobiles

In [13]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np
from itertools import zip_longest


In [14]:
def bs4_soup(pages):
    URL = f'https://www.flipkart.com/search?q=realme+mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={pages}'
    request = requests.get(URL)
    soup = bs(request.text,'html.parser')
    soup = soup.find('div',class_ = '_1YokD2 _3Mn1Gg')

    return soup , URL

In [15]:
def mob_specs(soup):
    names = soup.find_all('div', class_='_4rR01T')
    mob_names_list = [i.text if i else None for i in names]

    ratings = soup.find_all('div', class_='_3LWZlK')
    mob_ratings = [i.text if i else None for i in ratings]

    price = soup.find_all('div', class_='_30jeq3 _1_WHN1')
    mob_price = [i.text if i else None for i in price]

    img = soup.find_all('div', class_='CXW8mj')
    mob_img_URL = [i.find('img')['src'] if i.find('img') else None for i in img]

    # Find the maximum length among all lists
    max_length = max(len(mob_names_list), len(mob_ratings), len(mob_price), len(mob_img_URL))

    # Use zip_longest to ensure all lists have the same length
    mob_names = list(zip_longest(mob_names_list, [None] * max_length,fillvalue=np.nan))
    mob_ratings = list(zip_longest(mob_ratings, [None] * max_length,fillvalue=np.nan))
    mob_price = list(zip_longest(mob_price, [None] * max_length,fillvalue=np.nan))
    mob_img_URL = list(zip_longest(mob_img_URL, [None] * max_length,fillvalue=np.nan))

    return mob_names, mob_ratings, mob_price, mob_img_URL

In [16]:
import pandas as pd
from itertools import zip_longest

def features_from_mob_links(soup):
    mobile_links = ['https://www.flipkart.com' + i['href'] for i in soup.find_all('a', class_='_1fQZEK')]
    mobile_req = [requests.get(url=i) for i in mobile_links]
    mobile_soup = [bs(i.text, 'html.parser').find('div', class_='_1YokD2 _2GoDe3') for i in mobile_req]
    mobile_specs_soup = [i.find_all('div', class_='_3k-BhJ') for i in mobile_soup]

    storage_ram = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Memory' in str(specs_.text) and 'Storage' in str(specs_.text)]
    os_processor = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Os' in str(specs_.text) and 'Processor' in str(specs_.text)]
    camera = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Camera' in str(specs_.text) and 'FeaturesPrimary' in str(specs_.text)]
    display = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Display' in str(specs_.text) and 'FeaturesDisplay' in str(specs_.text)]
    network = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Connectivity' in str(specs_.text) and 'FeaturesNetwork' in str(specs_.text)]
    battery = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Battery' in str(specs_.text) and 'FeaturesBattery' in str(specs_.text)]

    # Find the maximum length among all lists
    max_length = max(len(storage_ram), len(os_processor), len(camera), len(display), len(network), len(battery))

    # Use zip_longest to ensure all lists have the same length
    storage_ram = list(zip_longest(storage_ram, [None] * max_length,fillvalue=np.nan))
    os_processor = list(zip_longest(os_processor, [None] * max_length,fillvalue=np.nan))
    camera = list(zip_longest(camera, [None] * max_length,fillvalue=np.nan))
    display = list(zip_longest(display, [None] * max_length,fillvalue=np.nan))
    network = list(zip_longest(network, [None] * max_length,fillvalue=np.nan))
    battery = list(zip_longest(battery, [None] * max_length,fillvalue=np.nan))

    return mobile_specs_soup, storage_ram, os_processor, camera, display, network, battery


In [17]:
df = pd.DataFrame()

page_start = 1

for pages in range(page_start, 20):

    soup, URL = bs4_soup(pages=pages)
    mob_names, mob_ratings, mob_price, mob_img_URL = mob_specs(soup)
    mobile_specs_soup, storage_ram, os_processor, camera, display, network, battery = features_from_mob_links(soup=soup)


    if mobile_specs_soup is not None and soup is not None:
        
        print(f"Length of mob_names: {len(mob_names)}")
        print(f"Length of mob_ratings: {len(mob_ratings)}")
        print(f"Length of mob_price: {len(mob_price)}")
        print(f"Length of mob_img_URL: {len(mob_img_URL)}")
        print(f"Length of storage_ram: {len(storage_ram)}")
        print(f"Length of os_processor: {len(os_processor)}")
        print(f"Length of camera: {len(camera)}")
        print(f"Length of display: {len(display)}")
        print(f"Length of network: {len(network)}")
        print(f"Length of battery: {len(battery)}")

        names = [i for i in mob_names]
        ratings = [i for i in mob_ratings]
        price = [i for i in mob_price]
        imgURL = [i for i in mob_img_URL]
        storage_ram_ = [i for i in storage_ram]
        cam = [i for i in camera]
        os_pro = [i for i in os_processor]
        dis = [i for i in display]
        net = [i for i in network]
        bat = [i for i in battery]

        data_dict = {
            'name': names,
            'ratings': ratings,
            'price': price,
            'imgURL': imgURL,
            'storage_ram': storage_ram_,
            'camera': cam,     
            'oS_Processor': os_pro,    
            'display': dis,
            'network': net,     
            'battery': bat,    
        }

        page_df = pd.DataFrame(data=data_dict)

        if not page_df.empty:
            df = pd.concat(objs=[df, page_df], ignore_index=True)
            print(f'scraping data from page:{pages}')
            print(f'working in {URL} Page')
            print(f'page df : page_shape : {page_df.shape} \n {page_df.sample(2)}')
            print(f'df : df shape : {df.shape} \n {df.sample(2)}')
            print()

    else:
        print(f"Skipping page {pages} due to empty mobile_specs_soup or soup.")


Length of mob_names: 24
Length of mob_ratings: 24
Length of mob_price: 24
Length of mob_img_URL: 24
Length of storage_ram: 24
Length of os_processor: 24
Length of camera: 24
Length of display: 24
Length of network: 24
Length of battery: 24
scraping data from page:1
working in https://www.flipkart.com/search?q=realme+mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1 Page
page df : page_shape : (24, 10) 
                                               name      ratings   
8          (realme C30 (Denim Black, 32 GB), None)  (4.1, None)  \
21  (realme 10 Pro 5G (Nebula Blue, 128 GB), None)  (4.3, None)   

              price                                             imgURL   
8    (₹6,499, None)  (https://rukminim2.flixcart.com/image/312/312/...  \
21  (₹18,999, None)  (https://rukminim2.flixcart.com/image/312/312/...   

                                          storage_ram   
8   (Internal Storage32 GBRAM3 GBExpandable Storag...  \
21  (Internal Sto

ValueError: All arrays must be of the same length

In [18]:
df

,name,ratings,price,imgURL,storage_ram,camera,oS_Processor,display,network,battery
0,"(realme C33 2023 (Sandy Gold, 128 GB), None)","(4.4, None)","(₹10,499, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM4 GBExpandable Stora...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 12Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
1,"(realme C33 2023 (Aqua Blue, 128 GB), None)","(4.4, None)","(₹10,499, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM4 GBExpandable Stora...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 12Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
2,"(realme C33 2023 (Night Sea, 128 GB), None)","(4.4, None)","(₹10,499, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM4 GBExpandable Stora...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 12Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
3,"(realme C30 (Denim Black, 32 GB), None)","(4.2, None)","(₹5,999, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage32 GBRAM2 GBExpandable Storag...,(Primary Camera AvailableYesPrimary Camera8MP ...,(Operating SystemAndroid 11Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
4,"(realme C30 (Bamboo Green, 32 GB), None)","(4.1, None)","(₹6,499, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage32 GBRAM3 GBExpandable Storag...,(Primary Camera AvailableYesPrimary Camera8MP ...,(Operating SystemAndroid 11Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
...,...,...,...,...,...,...,...,...,...,...
67,"(realme Narzo N55 (Prime Blue, 64 GB), None)","(4.2, None)","(₹12,899, None)",(https://rukminim2.flixcart.com/image/312/312/...,"(Internal Storage64 GBRAM4 GB, None)",(Primary Camera64MP Rear CameraDual Camera Len...,(Operating SystemAndroid 13Processor CoreOcta ...,(Display Size17.07 cm (6.72 inch)Resolution108...,(Network Type4G VOLTESupported Networks4G VoLT...,"(Battery Capacity5000 mAh, None)"
68,"(realme C33 (Aqua Blue, 64 GB), None)","(4.4, None)","(₹9,999, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage64 GBRAM4 GBExpandable Storag...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 12Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
69,"(realme C33 (Night Sea, 64 GB), None)","(4.4, None)","(₹9,999, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage64 GBRAM4 GBExpandable Storag...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 12Processor TypeUniso...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity5000 mAh, None)"
70,"(realme Narzo 50A (Oxygen Green, 128 GB), None)","(4.4, None)","(₹11,538, None)",(https://rukminim2.flixcart.com/image/312/312/...,(Internal Storage128 GBRAM4 GBExpandable Stora...,(Primary Camera AvailableYesPrimary Camera50MP...,(Operating SystemAndroid 11Processor TypeMedia...,(Display Size16.51 cm (6.5 inch)Resolution1600...,"(Network Type4G VOLTE, 4G, 3G, 2GSupported Net...","(Battery Capacity6000 mAh, None)"


In [3]:
print(os.getcwd())
os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem')
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem\notebook\data_Ingestion\scraping
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem
